In [0]:
%scala
dbutils.fs.mkdirs("/mnt/dados")

In [0]:
%python
dbutils.fs.ls('/mnt')

In [0]:
%python
dbutils.fs.ls('/mnt/dados')

In [0]:
%python
dbutils.fs.ls("/mnt/dados/inbound")

LENDO OS DADOS NA CAMADA INBOUND


In [0]:
val path = "dbfs:/mnt/dados/inbound/dados_brutos_imoveis.json"
val dados = spark.read.json(path)

In [0]:
display(dados)

**REMOVENDO COLUNAS**

In [0]:
val dados_anuncio = dados.drop("imagens","usuario")
display(dados_anuncio)

**CRIANDO UMA COLUNA **ID****

In [0]:
import org.apache.spark.sql.functions.col

In [0]:
val df_bronze = dados_anuncio.withColumn("id", col("anuncio.id"))
display(df_bronze)

**SALVANDO NA CAMADA BRONZE**

In [0]:
val path = "dbfs:/mnt/dados/bronze/dataset_imoveis"
df_bronze.write.format("delta").mode(SaveMode.Overwrite).save(path)

**LENDO OS DADOS NA CAMADA BRONZE**

In [0]:
%python
dbutils.fs.ls("/mnt/dados/bronze")

In [0]:
val path = "dbfs:/mnt/dados/bronze/dataset_imoveis/"
val df = spark.read.format("delta").load(path)

In [0]:
display(df)

In [0]:
display(df.select("anuncio.*"))

In [0]:
display(
  df.select("anuncio.*", "anuncio.endereco.*")
)

In [0]:
val dados_detalhados = df.select("anuncio.*", "anuncio.endereco.*")

**Removendo colunas**

In [0]:
val df_silver = dados_detalhados.drop("caracteristicas", "endereco")
display(df_silver)

**Salvando na camada silver**

In [0]:
val path = "dbfs:/mnt/dados/silver/dataset_imoveis"
df_silver.write.format("delta").mode("overwrite").save(path)